In [1]:
import torch
import numpy as np
import wandb

from utils import parse_config, unflatten_dot
from dataset import get_loader
from learner import Learner
from models import ASTPretrainedBigHead, ASTPretrainedSmallHead, RNN, ASTWithWeightedLayerPooling
from types import SimpleNamespace

/home/kpintaric/lumen-irmas/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kpintaric/lumen-irmas/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/kpintaric/lumen-irmas/venv/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
SEED = 123
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [38]:
def main():
    
    wandb.init()
    config = wandb.config
    train_dl = get_loader(config, subset="train")
    valid_dl = get_loader(config, subset="valid")
    
    model = ASTPretrainedSmallHead(n_classes=11)

    learn = Learner(train_dl, valid_dl, model, config)

    learn.fit(4)

In [39]:
import yaml
CONFIG_PATH = "./sweep_config.yaml"
with open(CONFIG_PATH) as file:
    config = yaml.safe_load(file)

sweep_id = wandb.sweep(sweep=config)
wandb.agent(sweep_id, function=main, count=1)